In [ ]:
# Estrategia 2 : Construir un dataset donde el numero de observaciones 0 y el numero de
# observaciones 1 sean esten en proporcion de 2 a 1 : De las observaciones con valor 0 (39922)
# vamos a muestrear 10000 de ellas

In [ ]:
# Estrategia 3 : Construir datos sinteticos para las observaciones con un
# numero de elementos (variable dependiente)


In [ ]:
# PAra mejorar el poder predictivo de una modelo de naturaleza bosque aleatorio.
# Consideraremos utilizar las variables de naturaleza cualitativa . Replicaremos
# el analisis realizado con la Estrategia 1 (Estrategia 2)

In [1]:
# Modulos basicos
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# cargamos la funcion train_test_split
from sklearn.model_selection import train_test_split

# carguemos el submodulo metrics
from sklearn import metrics

# Construyamos un bosque aleatorio para nuestro un problema de clasificacion
from sklearn.ensemble import RandomForestClassifier

# Dataset
df = pd.read_csv("bank-full.csv",sep = ";")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45211 entries, 0 to 45210
Data columns (total 17 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   age        45211 non-null  int64 
 1   job        45211 non-null  object
 2   marital    45211 non-null  object
 3   education  45211 non-null  object
 4   default    45211 non-null  object
 5   balance    45211 non-null  int64 
 6   housing    45211 non-null  object
 7   loan       45211 non-null  object
 8   contact    45211 non-null  object
 9   day        45211 non-null  int64 
 10  month      45211 non-null  object
 11  duration   45211 non-null  int64 
 12  campaign   45211 non-null  int64 
 13  pdays      45211 non-null  int64 
 14  previous   45211 non-null  int64 
 15  poutcome   45211 non-null  object
 16  y          45211 non-null  object
dtypes: int64(7), object(10)
memory usage: 5.9+ MB


In [2]:
# Eliminemos la columna day
df = df.drop("day", axis = 1)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45211 entries, 0 to 45210
Data columns (total 16 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   age        45211 non-null  int64 
 1   job        45211 non-null  object
 2   marital    45211 non-null  object
 3   education  45211 non-null  object
 4   default    45211 non-null  object
 5   balance    45211 non-null  int64 
 6   housing    45211 non-null  object
 7   loan       45211 non-null  object
 8   contact    45211 non-null  object
 9   month      45211 non-null  object
 10  duration   45211 non-null  int64 
 11  campaign   45211 non-null  int64 
 12  pdays      45211 non-null  int64 
 13  previous   45211 non-null  int64 
 14  poutcome   45211 non-null  object
 15  y          45211 non-null  object
dtypes: int64(6), object(10)
memory usage: 5.5+ MB


In [5]:
# Trabajemos con las variables de naturaleza cualitativa (object) que
# no sea la variable dependiente (y)
data_cuali = df.select_dtypes(include = "object").drop("y", axis = 1)
data_cuali.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45211 entries, 0 to 45210
Data columns (total 9 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   job        45211 non-null  object
 1   marital    45211 non-null  object
 2   education  45211 non-null  object
 3   default    45211 non-null  object
 4   housing    45211 non-null  object
 5   loan       45211 non-null  object
 6   contact    45211 non-null  object
 7   month      45211 non-null  object
 8   poutcome   45211 non-null  object
dtypes: object(9)
memory usage: 3.1+ MB


In [11]:
# Numero de niveles para cada una de las variables del objeto data_cuali
Dict_data_cuali = {}.fromkeys(data_cuali.columns)

for col in data_cuali.columns:
  Dict_data_cuali[col] = data_cuali[col].nunique()

pd.DataFrame.from_dict(Dict_data_cuali, orient = "index", columns =["NumUniques"]).sort_values(by = "NumUniques")

,NumUniques
default,2
housing,2
loan,2
marital,3
contact,3
education,4
poutcome,4
job,12
month,12


In [12]:
# COnstruyamos columnas dummies para data_cuali
for col in data_cuali.columns:
  Col4Dummy = data_cuali.pop(col)
  data_cuali = pd.concat([data_cuali, pd.get_dummies(Col4Dummy, prefix = col, dtype = int)], axis = 1)

data_cuali.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45211 entries, 0 to 45210
Data columns (total 44 columns):
 #   Column               Non-Null Count  Dtype
---  ------               --------------  -----
 0   job_admin.           45211 non-null  int64
 1   job_blue-collar      45211 non-null  int64
 2   job_entrepreneur     45211 non-null  int64
 3   job_housemaid        45211 non-null  int64
 4   job_management       45211 non-null  int64
 5   job_retired          45211 non-null  int64
 6   job_self-employed    45211 non-null  int64
 7   job_services         45211 non-null  int64
 8   job_student          45211 non-null  int64
 9   job_technician       45211 non-null  int64
 10  job_unemployed       45211 non-null  int64
 11  job_unknown          45211 non-null  int64
 12  marital_divorced     45211 non-null  int64
 13  marital_married      45211 non-null  int64
 14  marital_single       45211 non-null  int64
 15  education_primary    45211 non-null  int64
 16  education_secondary  4

In [14]:
#
data = pd.concat([df.select_dtypes(include = ["int64"]), data_cuali , df.y.map({"yes": 1, "no":0})], axis = 1)
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45211 entries, 0 to 45210
Data columns (total 51 columns):
 #   Column               Non-Null Count  Dtype
---  ------               --------------  -----
 0   age                  45211 non-null  int64
 1   balance              45211 non-null  int64
 2   duration             45211 non-null  int64
 3   campaign             45211 non-null  int64
 4   pdays                45211 non-null  int64
 5   previous             45211 non-null  int64
 6   job_admin.           45211 non-null  int64
 7   job_blue-collar      45211 non-null  int64
 8   job_entrepreneur     45211 non-null  int64
 9   job_housemaid        45211 non-null  int64
 10  job_management       45211 non-null  int64
 11  job_retired          45211 non-null  int64
 12  job_self-employed    45211 non-null  int64
 13  job_services         45211 non-null  int64
 14  job_student          45211 non-null  int64
 15  job_technician       45211 non-null  int64
 16  job_unemployed       4

In [15]:
# Guardemos en disco duro este nuevo dataframe
data.to_csv("data_bank_full.csv")

In [ ]:
# Implementar con este dataset la Estrategia 1 : Con todos los analisis realizados
# De igual manera implementar la Estrategia 2